In [1]:
import openpyxl
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import folium
from folium import plugins
from folium.plugins import HeatMap, MarkerCluster
from IPython.display import IFrame
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
from math import radians, sin, cos, sqrt, atan2
import branca
import requests

In [2]:
from src.data import orange_county_geometry

In [4]:
gdf = orange_county_geometry()

In [ ]:
import os
import geopandas as gpd

gdf = orange_county_geometry()

# san_diego_county_gpd = san_diego_county_gpd[['geometry']]

centroid = gdf.geometry.centroid.iloc[0]
map_center = [centroid.y, centroid.x]

# Create a Folium map
m = folium.Map(location=map_center, zoom_start=10)

# Add polygons to the map
for _, row in gdf.iterrows():
    folium.GeoJson(row.geometry).add_to(m)

# for _, row in san_diego_county_gpd.iterrows():
#     folium.GeoJson(row.geometry).add_to(m)



# Save map to an HTML file (optional)
m.save("congressional_map.html")

# Display map in Jupyter Notebook (optional)
m

# # Convert geometries to Shapely polygons
# polygons = gdf["geometry"].apply(lambda geom: geom if isinstance(geom, Polygon) else None)

# # Drop None values in case of non-polygon geometries
# polygons = polygons.dropna().tolist()

# # Print or use the first polygon
# print(polygons[0])

In [ ]:
from helper_functions import haversine_distance

In [ ]:
def get_elevation(lat, lon):
    url = f"https://api.open-elevation.com/api/v1/lookup?locations={lat},{lon}"
    response = requests.get(url).json()
    return response['results'][0]['elevation'] if "results" in response else None

def create_point(row):
    return Point(row['longitude'], row['latitude'])

In [5]:
# Load the Excel files into Pandas DataFrames
gis_weather_station = pd.read_csv('./data/raw//gis_weatherstation_shape_2024_10_04.csv')
windspeed_snapshot = pd.read_csv('./data/raw/src_wings_meteorology_windspeed_snapshot_2023_08_02.csv')

src_vri_snapshot = pd.read_csv('./data/raw/src_vri_snapshot_2024_03_20.csv')
nam = pd.read_csv('./data/raw/nam.csv')
san_diego_county_gpd = gpd.read_file("data/raw/San_Diego_County_Boundary.geojson")

In [9]:
gis_weather_station['shape'].head()

0    MULTIPOLYGON Z (((-116.783579 32.826647 0, -11...
1    MULTIPOLYGON Z (((-117.297615 33.325552 0, -11...
2    MULTIPOLYGON Z (((-116.753684 32.879331 0, -11...
3    MULTIPOLYGON Z (((-116.988931 33.050367 0, -11...
4    MULTIPOLYGON Z (((-117.214118 33.377714 0, -11...
Name: shape, dtype: object

In [10]:
gis_weather_station['geometry'] = gis_weather_station['shape'].apply(wkt.loads)
gis_weather_station_gpd = gpd.GeoDataFrame(gis_weather_station, geometry='geometry', crs=f"EPSG:{gis_weather_station['shape_srid'][0]}")

In [ ]:
type(gis_weather_station_gpd['geometry'].iloc[0].x)

In [11]:
gis_weather_station['geometry'] = gis_weather_station['shape'].apply(wkt.loads)
gis_weather_station_gpd = gpd.GeoDataFrame(gis_weather_station, geometry='geometry', crs=f"EPSG:{gis_weather_station['shape_srid'][0]}")

src_vri_snapshot['geometry'] = src_vri_snapshot['shape'].apply(wkt.loads)
src_vri_snapshot_gpd = gpd.GeoDataFrame(src_vri_snapshot, geometry='geometry', crs=f"EPSG:{src_vri_snapshot['shape_srid'][0]}")

# Get ESPG:4326
nam_crs = src_vri_snapshot['shape_srid'][0]

# Create NAM Geodataframe
nam['geometry'] = nam.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
nam_gpd = gpd.GeoDataFrame(nam, geometry='geometry', crs=nam_crs)

print(f"Weather Station CRS:    {gis_weather_station_gpd.crs}")
print(f"VRI Polygon CRS:        {src_vri_snapshot_gpd.crs}")
print(f"NAM CRS:                {nam_gpd.crs}")
print(f"San Diego County:       {san_diego_county_gpd.crs}")

Weather Station CRS:    EPSG:4431
VRI Polygon CRS:        EPSG:4326
NAM CRS:                EPSG:4326
San Diego County:       EPSG:4326


In [ ]:
gis_weather_station_gpd = gis_weather_station_gpd.to_crs(src_vri_snapshot_gpd.crs)

print(f"Weather Station CRS:    {gis_weather_station_gpd.crs}")
print(f"VRI Polygon CRS:        {src_vri_snapshot_gpd.crs}")
print(f"NAM CRS:                {nam_gpd.crs}")
print(f"San Diego County:       {san_diego_county_gpd.crs}")

In [ ]:
# Get elevation and nearest coastline for Weather Station
gis_weather_station_gpd['elevation_m'] = gis_weather_station_gpd.apply(lambda row: get_elevation(row['geometry'].y, row['geometry'].x), axis=1)

# Save file as CSV
gis_weather_station_gpd.to_csv("../../data/modified/gis_weather_station_with_elevation.csv", index=False)

In [ ]:
san_diego_county_gpd = san_diego_county_gpd[['geometry']]

In [ ]:
# Filter NAM within the VRI polygons
nam_vri_gpd = gpd.sjoin(nam_gpd, src_vri_snapshot_gpd, how='inner', predicate='within').drop(columns=['index_right'])
nam_san_diego_county_gpd = gpd.sjoin(nam_gpd, san_diego_county_gpd, how='inner', predicate='within').drop(columns=['index_right'])
nam_vri_san_diego_county_gpd = gpd.sjoin(nam_san_diego_county_gpd, src_vri_snapshot_gpd, how='inner', predicate='within')

In [ ]:
print(f"Number of Unique Points within VRI Polygon: {nam_vri_gpd[['geometry']].nunique()}")
print(f"Number of Unique Points within San Diego County: {nam_san_diego_county_gpd[['geometry']].nunique()}")
print(f"Number of Unique Points within VRI Polygon in San Diego County: {nam_vri_san_diego_county_gpd[['geometry']].nunique()}")
print(f"Number of Unique Points within VRI Polygon and Outside San Diego County: {nam_vri_gpd[['geometry']].nunique() - nam_vri_san_diego_county_gpd[['geometry']].nunique()}")

print(f"Shape of NAM data within VRI polgyon: {nam_vri_gpd.shape}")
print(f"Shape of NAM data within San Diego county: {nam_san_diego_county_gpd.shape}")

In [ ]:
nam_vri_gpd = nam_vri_gpd[['latitude', 'longitude', 'date', 'average_wind_speed', 'geometry']]

nam_vri_san_diego_county_gpd_combined = gpd.GeoDataFrame(pd.concat([nam_vri_gpd, nam_san_diego_county_gpd], ignore_index=True))
nam_vri_san_diego_county_gpd_combined = nam_vri_san_diego_county_gpd_combined.drop_duplicates()

print(f"Number of Unique Points within all VRI Polygon and San Diego County: {nam_vri_san_diego_county_gpd_combined[['geometry']].nunique()}")
print(f"Shape of NAM data within all VRI Polygon and San Diego County: {nam_vri_san_diego_county_gpd_combined.shape}")

In [ ]:
# Get the unique geometries within the VRI
nam_vri_san_diego_county_gpd_combined_geometry = nam_vri_san_diego_county_gpd_combined[['geometry']].drop_duplicates().reset_index(drop=True)
nam_vri_san_diego_county_gpd_combined_geometry['nam_elevation_m'] = nam_vri_san_diego_county_gpd_combined_geometry.apply(lambda row: get_elevation(row['geometry'].y, row['geometry'].x), axis=1)
nam_vri_san_diego_county_gpd_combined_geometry.head()

In [ ]:
# Add elevation to NAM data
nam_vri_san_diego_county_gpd_combined = nam_vri_san_diego_county_gpd_combined.merge(nam_vri_san_diego_county_gpd_combined_geometry, on='geometry', how='inner')
nam_vri_san_diego_county_gpd_combined.head()

In [ ]:
# # Save file as CSV
nam_vri_san_diego_county_gpd_combined.to_csv("data/nam_with_elevation.csv")

In [ ]:
import folium
from shapely.geometry import Polygon
import geopandas as gpd

# Define the refined Southern Orange County polygon
refined_southern_oc_coords = [
    (-117.7, 33.5),  # Laguna Hills area
    (-117.65, 33.55),  # Mission Viejo area
    (-117.6, 33.5),  # Laguna Niguel area
    (-117.55, 33.45),  # Capistrano area
    (-117.55, 33.4),  # Dana Point coast
    (-117.6, 33.35),  # San Onofre coast
    (-117.65, 33.3),  # Las Pulgas (Camp Pendleton border)
    (-117.7, 33.35),  # Inland near Camp Pendleton
    (-117.7, 33.5)   # Closing the polygon
]

# Create the polygon
southern_oc_polygon = Polygon(refined_southern_oc_coords)

# Create a Folium map centered around Southern Orange County
m = folium.Map(location=[33.45, -117.6], zoom_start=10)

# Convert polygon to a GeoDataFrame for plotting
gdf = gpd.GeoDataFrame([1], geometry=[southern_oc_polygon], crs="EPSG:4326")

# Add the polygon to the map
folium.GeoJson(gdf, name="Southern Orange County Service Area").add_to(m)

# Display map
m
